In [43]:
from PIL import Image
import requests
from io import BytesIO
import dlib
from numpy import asarray
import os.path
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import dlib
import os

In [181]:
def detect_face(url, default_max_size=800,size = 300, padding = 0.25):
    cnn_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
    sp = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')
    base = 2000  # largest width and height
    
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = asarray(img)

    old_height, old_width, _ = img.shape
    if old_width > old_height:
        new_width, new_height = default_max_size, int(default_max_size * old_height / old_width)
    else:
        new_width, new_height =  int(default_max_size * old_width / old_height), default_max_size
    img = dlib.resize_image(img, rows=new_height, cols=new_width)

    dets = cnn_face_detector(img, 1)
    num_faces = len(dets)
    if num_faces == 0:
        return []
    # Find the 5 face landmarks we need to do the alignment.
    faces = dlib.full_object_detections()
    for detection in dets:
        rect = detection.rect
        faces.append(sp(img, rect))
    faces = dlib.get_face_chips(img, faces, size=size, padding = padding)

    return faces

In [176]:
def predict_age_gender_race(url):
    
    model = 'res34_fair_align_multi_7_20190809.pt' 
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model_fair_7 = torchvision.models.resnet34(pretrained=True)
    model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 18)
    map_location=torch.device('cpu')
    model_fair_7.load_state_dict(torch.load(model, map_location=map_location))
    model_fair_7 = model_fair_7.to(device)
    model_fair_7.eval()

    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    race_preds_fair = []
    gender_preds_fair = []
    age_preds_fair = []

    faces = detect_face(url)
    if len(faces) == 0:
        return pd.DataFrame(columns=['race', 'gender', 'age'], dtype=object)
    for face in faces:
        
        image = trans(face)
        image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
        image = image.to(device)

        # fair
        outputs = model_fair_7(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)
        
        race_outputs = outputs[:7]
        gender_outputs = outputs[7:9]
        age_outputs = outputs[9:18]
        
        race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
        gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
        age_score = np.exp(age_outputs) / np.sum(np.exp(age_outputs))

        race_pred = np.argmax(race_score)
        gender_pred = np.argmax(gender_score)
        age_pred = np.argmax(age_score)

        race_preds_fair.append(race_pred)
        gender_preds_fair.append(gender_pred)
        age_preds_fair.append(age_pred)
    
    result = pd.DataFrame([race_preds_fair,
                           gender_preds_fair,
                           age_preds_fair,
                           ]).T
    
    result.columns = ['race_preds_fair',
                      'gender_preds_fair',
                      'age_preds_fair']
    
    result.loc[result['race_preds_fair'] == 0, 'race'] = 'White'
    result.loc[result['race_preds_fair'] == 1, 'race'] = 'Black'
    result.loc[result['race_preds_fair'] == 2, 'race'] = 'Latino_Hispanic'
    result.loc[result['race_preds_fair'] == 3, 'race'] = 'East Asian'
    result.loc[result['race_preds_fair'] == 4, 'race'] = 'Southeast Asian'
    result.loc[result['race_preds_fair'] == 5, 'race'] = 'Indian'
    result.loc[result['race_preds_fair'] == 6, 'race'] = 'Middle Eastern'

    # gender
    result.loc[result['gender_preds_fair'] == 0, 'gender'] = 'Male'
    result.loc[result['gender_preds_fair'] == 1, 'gender'] = 'Female'

    # age
    result.loc[result['age_preds_fair'] <= 1, 'age'] = '0-9'
    result.loc[result['age_preds_fair'] == 2, 'age'] = '10-19'
    result.loc[result['age_preds_fair'] == 3, 'age'] = '20-29'
    result.loc[result['age_preds_fair'] == 4, 'age'] = '30-39'
    result.loc[result['age_preds_fair'] == 5, 'age'] = '40-49'
    result.loc[result['age_preds_fair'] == 6, 'age'] = '50-59'
    result.loc[result['age_preds_fair'] == 7, 'age'] = '60-69'
    result.loc[result['age_preds_fair'] == 8, 'age'] = '70+'

    return result[['race', 'gender', 'age']]

In [185]:
urls = ['https://images2.persgroep.net/rcs/MKddFc-N7DUGurfPhDdqQRjSqNo/diocontent/211823231/_fit/500/400?appId=c66748603e84b2e1a9bbfcdd8de0f671',
       "https://images0.persgroep.net/rcs/ivTpKVc2V_XGz1yT_VQudueLCG0/diocontent/180351536/_fit/500/400?appId=c66748603e84b2e1a9bbfcdd8de0f671",
        "https://images0.persgroep.net/rcs/xkiDi4GwyijxeOYsO3pzbtjIm4Y/diocontent/212477697/_crop/674/747/4936/2468/_fill/280/140?appId=93a17a8fd81db0de025c8abd1cca1279&quality=0.85"]

dlib.DLIB_USE_CUDA = True

for url in urls:
    print(predict_age_gender_race(url))


    race  gender    age
0  White  Female  20-29
              race  gender    age
0  Latino_Hispanic  Female  40-49
    race gender    age
0  Black   Male  20-29
1  White   Male  40-49
